# Analysis of ALMA NGC 253 Data for Density Tracing Species

## (1) Importing Python packages you will be using

In [2]:
#Use this window to import packages you will use later
#When you first start working or import a new package, this cell needs to be run before doing anything else.


import numpy as np                          # I use this for maths and sometimes arrays.
                                            # "np" is just an abbreviation since we call this package so often!
import pylab                                # I use this for arrays and plotting
import matplotlib                           # Another plotting package
import matplotlib.gridspec as gridspec      # If there is a task you use a lot, importing it like this 
                                            # keeps you from having to constantly type "matplotlib.gridspec"
                                            # every time you call that task!
import scipy

import astropy.io.fits as fits              # I use this to import FITS images. 
                                            # Astropy should be included in your conda installation
from astropy.wcs import WCS                 # This one helps us add coordinates to images
from astropy import units as u              # This one helps with unit conversion
from astropy import constants as const

import regions

import pyspeckit as psk                     # I use this to do spectral line fitting
                                            # You probably don't have it installed; 
                                            # to install, type 'pip install pyspeckit' in a terminal window
        
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes

from reproject import reproject_interp      # Reproject is another useful package you should install
from reproject.mosaicking import find_optimal_celestial_wcs 


                                            # Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

                                            # Super important!! 
                                            # This embeds plots in the Jupyter window 
                                            # (instead of showing them as pop-ups)
%matplotlib inline                             

matplotlib.rc('font', family='sans-serif')  # Set plot fonts
matplotlib.rc('text', usetex=True)      
#matplotlib.rc('text', usetex=False)        # This is a quick fix if you don't have latex on your computer

In [3]:
#path = '/Users/ashleylieber/MIlls_Research/NGC253_AlmaData/HCN4-3.fits'
path = '/Users/ashleylieber/MIlls_Research/NGC253_AlmaData/HCN_J1-0.cube.fits'
                                # This string is not just the name of a file, but its full address.
                                # If, like this file, it is in the same directory as your notebook,
                                # you can specify just the filename. If it lives elsewhere you need
                                # to give a relative (e.g., data/name.fits or ../directory/name.fits)
                                # or absolute (e.g., EACMills/Project/Data/name.fits) path to the file.


                                # The fits images we will work with consist of two main parts:
                                # (1) An array of 2-4 dimensions that holds the value of each image pixel 
                                # (2) A "header" which contains a summary of information associated with this array                
          

image = fits.getdata(path)      # This command reads in the data array part of the fits file, so we can manipulate it 
                                # like any other python array
    
header = fits.getheader(path)   # This command reads in the header, which consists of a series of variables 
                                # and associated values  

In [4]:
#eader

In [5]:
hi_data = fits.open(path)  # Open the FITS file for reading
cube = SpectralCube.read(hi_data)  # Initiate a SpectralCube
hi_data.close() 

In [6]:
print(cube)

SpectralCube with shape=(180, 1221, 1221) and unit=Jy / beam:
 n_x:   1221  type_x: RA---SIN  unit_x: deg    range:    11.881963 deg:   11.893582 deg
 n_y:   1221  type_y: DEC--SIN  unit_y: deg    range:   -25.293789 deg:  -25.283283 deg
 n_s:    180  type_s: VRAD      unit_s: m / s  range:   -27095.993 m / s:  564232.971 m / s


## Cropping

In [9]:
region_str = "circle[11.8875deg,-25.2886deg], 0.008deg], coord=fk5"
region = regions.Regions.parse(region_str, format='crtf')

In [10]:
sc_example = SpectralCube.read(path)
crop = sc_example.subcube_from_regions(region)